In [2]:
import os, sys
from datetime import datetime
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, MultiLineString
import numpy as np
from arcgis.geometry._types import Polyline as esriPolyline
from arcgis.geometry._types import Polygon as esriPolygon
from arcgis.gis import GIS
from arcgis.features import GeoAccessor
from arcgis.features.analysis import join_features
from arcgis import features
from getpass import getpass
import json
import requests
from joblib import Parallel, delayed
import swifter
from glob import glob

In [3]:
def calcEditCodes(hucid, df):
    print(f"Starting on {hucid}")
    df.reset_index(inplace=True, drop=True)
    # Grab ADWR and NHD data within the same HUC
    adwr_df = adwr_huc12[adwr_huc12.HUC12 == hucid]
    nhd_df = nhdFlowlines_huc12[nhdFlowlines_huc12.HUC12 == hucid]
    if len(adwr_df) == 0:
        print(f"no matching adwr watersheds for {hucid}")
        return None
    elif len(nhd_df) == 0:
        print(f"no matching nhd watersheds for {hucid}")
        return None
    #print(len(nhd_df), len(adwr_df))
    #df[["EditCode","EditNote"]] = df.apply(lambda r: findIdentical(r, nhd_df, adwr_df), axis=1)# if r.PCLength == 0 else pd.Series([3,note]), axis=1)
    df[["localNHD_idx","localADWR_idx"]] = df.apply(lambda r: findIdentical(r, nhd_df, adwr_df), axis=1)# if r.PCLength == 0 else pd.Series([3,note]), axis=1)
    return df


def findLocalByLength(geom, ds, distance=10):
    """return list of stream lengths in ds where within distance value (meters, default 10)
    and also whose length is within +- 15% of target length. The stream with maximum length
    is returned"""
    
    t_length = geom.length
    geom_b = geom.buffer(distance)
    ds_l = ds[ds.intersects(geom_b)]
    if len(ds_l) == 0:
        return None
        #return False
    ds_l_c = gpd.clip(ds_l, geom_b)
    ds_l_c["Length"] = round(ds_l_c.geometry.length)
    
    # This checks for only identical.
    #ds_l_c = ds_l_c[ds_l_c["Length"] == round(geom.length)]
    
    # +- tolerance of 15%
    geom_lowlim = t_length * 0.85
    geom_highlim = t_length * 1.15
    ds_l_c = ds_l_c[(ds_l_c["Length"] >= geom_lowlim) & (ds_l_c["Length"] <= geom_highlim)]
    ds_l_c["LengthDiff"] = ds_l_c.Length.apply(lambda l: abs(l-t_length))
    ds_l_c.sort_values(by="LengthDiff", ascending=True, inplace=True)
    #lengths = ds_l_c["Length"].values.tolist()
    featIdxs = ds_l_c["Huc12FeatIdx"].values.tolist()
    
    return ",".join([str(fidx) for fidx in featIdxs])#lengths[:3]

    """
    if len(lengths) == 1:
        # only one match, so return the value
        return ds_l_c["Length"].values[0]
        #return True
    else:
        return 0
        #return False
    """

def findIdentical(row, nhdDF, adwrDF):
    idx = row.name
    if idx % 10000 == 0 and idx != 0:
        print(f"On {idx} - {datetime.now()}")
    geom = row.geometry
    nhd_local_idx = findLocalByLength(geom, nhdDF)
    adwr_local_idx = findLocalByLength(geom, adwrDF)
    
    return pd.Series([nhd_local_idx, adwr_local_idx])
    #if nhd_local and adwr_local:
    #    return pd.Series([0, note]) 
    #else:
    #    return pd.Series([3, note]) 
    
    
def calcSimilarity(row):
    adeqLength = row.ADEQLength
        
    lengths = [row.NHDLength, row.NHDLength, row.ADWRLength, row.PCLength]
    lengths.remove(0) if 0 in lengths else lengths
    simScores = []
    totalAllLengths = np.sum(lengths)
    #print("lengths", lengths)
    for length in lengths:
        absDiff = abs(adeqLength - length)
        #print("AbsDiff", absDiff)
        simScore = absDiff/adeqLength
        #print("simScore", simScore)
        lengthWeight = length/totalAllLengths
        simScoreWeight = simScore * lengthWeight
        simScores.append(simScoreWeight)
    
    simScoreF = 1-np.mean(simScores)
    
    return simScoreF

    

    

def toGeodataFrame(sdf):
    geomtype = [key for key in sdf.SHAPE.values[0].keys()][0]
    if geomtype == "rings":
        gType = esriPolygon
    elif geomtype == "paths":
        gType = esriPolyline
    sdf["geometry"] = sdf.SHAPE.apply(lambda g: MultiLineString(g[geomtype]) if type(g) is gType else np.NaN)
    sdf = sdf[sdf["geometry"].notnull()]
    crs = sdf["SHAPE"].values[0]["spatialReference"]["latestWkid"]
    sdf_gdf = gpd.GeoDataFrame(sdf, geometry="geometry", crs=f"epsg:{crs}")
    return sdf_gdf




def joinDFtoHUC12(flowline_df, huc12):
    """Overlaying all is very slow with GEOS, so spatial join
    first, then find multiple joins (e.g. 2 to 1) and only overlay those"""
    huc12 = huc12[["HUC12", "geometry"]]
    
    flowline_df.reset_index(inplace=True)
    flowline_df.rename(columns={"index":"FeatIdx"}, inplace=True)
    
    flowline_df_sjoin = gpd.sjoin(flowline_df, huc12, lsuffix="")
    
    incolumns = flowline_df.columns.tolist()

    flowlinedf_multi_join = flowline_df_sjoin[flowline_df_sjoin.FeatIdx.duplicated(keep=False)].copy()
    flowlinedf_single_join = flowline_df_sjoin[~flowline_df_sjoin.FeatIdx.duplicated(keep=False)].copy()

    # only keep columns that were in original before sjoin
    flowlinedf_multi_join = flowlinedf_multi_join[incolumns]
    # drop the duplicated feature resulting from the sjoin
    flowlinedf_multi_join.drop_duplicates(subset="FeatIdx", keep="first", inplace=True)
    # overlay on huc12s to split features
    flowlinedf_multi_join = gpd.overlay(flowlinedf_multi_join, huc12, how='intersection')
    
    #Create Huc12FeatIdx after overlay so one unique for each flowline in each huc
    flowline_df.reset_index(inplace=True)
    flowline_df.rename(columns={"index":"Huc12FeatIdx"}, inplace=True)

    outcolumns = incolumns + ["HUC12"]

    # merge 1-1 join features and split features back together
    flowlines_df_overlay = pd.concat([flowlinedf_single_join[outcolumns], flowlinedf_multi_join[outcolumns]])
    
    return flowlines_df_overlay


def getNHDData(nhdData_loc, huc4Num):
    folder = f"NHDPLUS_H_{huc4Num}_HU4_GDB"
    gdb = f"NHDPLUS_H_{huc4Num}_HU4_GDB.gdb"
    gdb_loc = os.path.join(nhdData_loc, folder, gdb)
    if not os.path.exists(gdb_loc):
        raise ValueError(f"Unable to find the gdb {gdb_loc}")
    df_watershed = gpd.read_file(gdb_loc, layer="WBDHU4")
    df_watershed = df_watershed[df_watershed.HUC4 == str(huc4Num)].to_crs("epsg:3857")
    df_flowlines = gpd.read_file(gdb_loc, layer="NHDFlowline").to_crs("epsg:3857")
    # should we filter by ftype 460 ("stream/river")?
    
    return df_watershed, df_flowlines


def getADWRLayer():
    
    azgeo = "https://azgeo.maps.arcgis.com/home/index.html"
    user = "bhickson_azgeo"
    if 'password' not in globals():
        password = getpass(f"Provide password for user {user}:")

    gis = GIS(azgeo, user, password)

    adwrItem = gis.content.get("ba7ac533920f47b4bbbb165772bbc7aa")
    
    dfs = []
    for i, layer in enumerate(adwrItem.layers):
        print(f"Getting layer {i}")
        adwr_sdf = layer.query().sdf
        print(f"Got layer {i}")
        dfs.append(toGeodataFrame(adwr_sdf))
    
    adwrLayers = pd.concat(dfs)
    adwrLayers = adwrLayers.explode()

    adwrLayers["PERMANENT_"] = adwrLayers.apply(lambda r: r.PERMANENT if pd.isnull(r.PERMANENT_) else r.PERMANENT_, axis=1)
    del adwrLayers["PERMANENT"], adwrLayers["SHAPE_LEN"], adwrLayers["SHAPE_LENG"], adwrLayers["SHAPE"]
    adwrLayers.rename({"PERMANENT_": "PERMANENT", "Shape__Length": "Shape_Length", "FID": "FeatureID"}, inplace=True)
    adwrLayers.FDATE = adwrLayers.FDATE.astype(str)
    
    adwrLayers.reset_index(drop=True, inplace=True)
    
    return adwrLayers

In [4]:
nhd_gdb = r"S:\BHickson\Shared\NHD\NHD_H_Arizona_State_GDB.gdb"
if not os.path.exists(nhd_gdb):
    nhd_gdb = r"./Data/NHD_H_Arizona_State_GDB/NHD_H_Arizona_State_GDB.gdb"

huc4s_t = gpd.read_file(nhd_gdb, layer="WBDHU4")
huc4s_t.head(3)

,TNMID,MetaSourceID,SourceDataDesc,SourceOriginator,SourceFeatureID,LoadDate,GNIS_ID,AreaAcres,AreaSqKm,States,HUC4,Name,Shape_Length,Shape_Area,geometry
0,{C7D39B24-68A5-482B-9EC1-B20FEB452850},None,None,None,None,2020-06-12T11:34:35,None,9650675.10,39054.93,AZ,1507,Lower Gila,13.944801,3.779091,"MULTIPOLYGON (((-112.15810 34.71632, -112.1580..."
1,{238D719A-43E5-473F-AB01-2EC0CDA6F168},None,None,None,None,2020-06-04T20:07:18,None,8714022.72,35264.43,"AZ,UT",1407,Upper Colorado-Dirty Devil,12.874770,3.605345,"MULTIPOLYGON (((-111.47581 39.12450, -111.4754..."
2,{6CD2178B-03B1-4D2D-9BC5-B1328A200221},None,None,None,None,2020-06-04T20:34:59,None,8622108.89,34892.47,AZ,1506,Salt,19.642615,3.417107,"MULTIPOLYGON (((-113.09819 35.86434, -113.0979..."


## READ IN NHD, ADWR, Arizona State, and HUC12s reproject to 3857 and clip HUCs to arizona

In [5]:
%%time
adwr_loc = "./Data/ADWR_Arizona.gpkg"

arizona = gpd.read_file("./Data/Arizona.gpkg")
huc12s = gpd.read_file(nhd_gdb, layer="WBDHU12")
nhdFlowlines = gpd.read_file(nhd_gdb, layer="NHDFlowline")

arizona.to_crs("epsg:3857", inplace=True)
huc12s.to_crs("epsg:3857", inplace=True)
nhdFlowlines.to_crs("epsg:3857", inplace=True)

huc12s_az = gpd.overlay(huc12s, arizona)

Wall time: 3min 32s


## Split NHD, ADEQ, AND ADWR to HUC12 Boundaries and Join to HUCs

In [7]:
adeq_huc12_loc = "./Data/ADEQ_Arizona_HUC12.gpkg"
adwr_huc12_loc = "./Data/ADEQ_Arizona_HUC12.gpkg"
nhd_huc12_loc = "./Data/NHDFlowlines_HUC12.gpkg"

if os.path.exists(adeq_huc12_loc):
    print("Reading in ADEQ Data with HUC12 Joined")
    adeq_huc12 = gpd.read_file("./Data/ADEQ_Arizona_HUC12.gpkg")
else:
    adeq_arizona = gpd.read_file(r"./Data/ADEQ_HR_Streams/NHD_HR_WBID.shp")
    adeq_arizona.to_crs("epsg:3857", inplace=True)
    adeq_arizona = adeq_arizona.explode()
    adeq_huc12 = joinDFtoHUC12(adeq_arizona, huc12s_az)
    adeq_huc12.to_file("./Data/ADEQ_Arizona_HUC12.gpkg", driver="GPKG")

    
if os.path.exists(adwr_huc12_loc):
    print("Reading in ADWR Data with HUC12 Joined")
    adwr_huc12 = gpd.read_file("./Data/ADWR_Arizona_HUC12.gpkg")
else:
    if not os.path.exists(adwr_loc):
        print("Pulling down ADWR Data")
        adwr_data = getADWRLayer(adwr_item)
        adwr_data.to_file(adwr_loc, driver="GPKG")
    else:
        print("Reading in ADWR Data")
        adwr_data = gpd.read_file(adwr_loc)

    adwr_data.to_crs("epsg:3857")
    adwr_huc12 = joinDFtoHUC12(adwr_data, huc12s_az)
    adwr_huc12.to_file("./Data/ADWR_Arizona_HUC12.gpkg", driver="GPKG")

if os.path.exists(nhd_huc12_loc):
    print(f"Reading in NHD Flowline Data with HUC12 Joined")
    nhdFlowlines_huc12 = gpd.read_file(nhd_huc12_loc)
else:
    print("Joining Arizona HUC12s to NHD Flowlines")
    nhdFlowlines_huc12 = joinDFtoHUC12(nhdFlowlines, huc12s_az)
    nhdFlowlines_huc12.to_file(nhd_huc12_loc, driver="GPKG")

Reading in ADEQ Data with HUC12 Joined
Reading in ADWR Data with HUC12 Joined
Reading in NHD Flowline Data with HUC12 Joined


## Create Unique ID (Huc12FeatIDx) after splitting by HUC by reseting Index for all NHD, ADEQ, and ADWR datasets

In [8]:
#Create Huc12FeatIdx after overlay so one unique for each flowline in each huc
for df in [adeq_huc12, adwr_huc12, nhdFlowlines_huc12]:
    if "Huc12FeatIdx" not in df:
        df.reset_index(inplace=True)
        df.rename(columns={"index":"Huc12FeatIdx"}, inplace=True)

### ADWR doesn't have watershed data over north-east Arizona. Pre filter ADEQ and NHD datasets so that we only compare areas where all three exist

In [9]:
adeq_huc12 = adeq_huc12[adeq_huc12.HUC12.isin(adwr_huc12.HUC12.unique().tolist())]
nhdFlowlines_huc12 = nhdFlowlines_huc12[nhdFlowlines_huc12.HUC12.isin(adwr_huc12.HUC12.unique().tolist())]

adwr_huc12 = adwr_huc12[adwr_huc12.HUC12.isin(adeq_huc12.HUC12.unique().tolist())]
nhdFlowlines_huc12 = nhdFlowlines_huc12[nhdFlowlines_huc12.HUC12.isin(adeq_huc12.HUC12.unique().tolist())]

## NHD AND ADWR have irrationally small (less than 10cm). Remove them

In [10]:
# remove some irrationally small linework
nhdFlowlines_huc12.Shape_Length = nhdFlowlines_huc12.geometry.length
adwr_huc12.Shape_Length = adwr_huc12.geometry.length

nhdFlowlines_huc12 = nhdFlowlines_huc12[nhdFlowlines_huc12.Shape_Length > 0.1]
adwr_huc12 = adwr_huc12[adwr_huc12.Shape_Length > 0.1]

print(nhdFlowlines_huc12.shape, adwr_huc12.shape)

(659957, 19) (481963, 16)


In [ ]:
#[calcEditCodes(huc12id, group) for huc12id, group in adeq_huc4_group.groupby("HUC12")]

## Compare NHD and ADWR to ADEQ and generate edit codes

In [25]:
%%time
tday = datetime.now().strftime("%Y-%m-%d")
note = f"EditCode as assessed by Ben's script on {tday}"

adeq_huc12["HUC4"] = adeq_huc12["HUC12"].apply(lambda huc12: huc12[:4])

files = []
os.makedirs(f"./Data/AssessmentByHUC4", exist_ok=True)
for huc4, adeq_huc4_group in adeq_huc12.groupby("HUC4"):
    print(f"Starting HUC4 {huc4} at {datetime.now()}")
    ofile = f"./Data/AssessmentByHUC4/adeq_huc12_lengthsComp_HUC4-{huc4}.gpkg"
    if os.path.exists(ofile):
        print(f"{ofile} exists")
        continue
    else:
        grps = adeq_huc4_group.groupby("HUC12")
        print(f"{len(grps)} total HUC12s in HUC4 {huc4}")
        huc4_calcs = Parallel(n_jobs=-1, verbose=5, max_nbytes=None)(delayed(calcEditCodes)(huc12id, group) for huc12id, group in adeq_huc4_group.groupby("HUC12"))
        adeq_edits_j_concat = pd.concat(huc4_calcs)
        adeq_edits_j_concat.to_file(ofile, driver="GPKG")
    files.append(ofile)

Starting HUC4 1408 at 2021-10-13 11:11:06.766307
2 total HUC12s in HUC4 1408


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:   53.6s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:   53.6s finished


Starting HUC4 1501 at 2021-10-13 11:12:00.747617
444 total HUC12s in HUC4 1501


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
C:\Users\BenJames\anaconda3\envs\arcgis\lib\site-packages\joblib\externals\loky\process_executor.py:703: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed: 14.1min
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed: 45.2min
[Parallel(n_jobs=-1)]: Done 256 tasks      | elapsed: 88.1min
[Parallel(n_jobs=-1)]: Done 444 out of 444 | elapsed: 153.0min finished


Starting HUC4 1502 at 2021-10-13 13:45:23.327533
636 total HUC12s in HUC4 1502


...

Starting HUC4 1507 at 2021-10-14 01:44:32.667469
378 total HUC12s in HUC4 1507


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed: 14.3min
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed: 44.0min
[Parallel(n_jobs=-1)]: Done 256 tasks      | elapsed: 86.6min
[Parallel(n_jobs=-1)]: Done 378 out of 378 | elapsed: 127.7min finished


Starting HUC4 1508 at 2021-10-14 03:52:51.483032
5 total HUC12s in HUC4 1508


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   51.2s remaining:  1.3min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.8min finished


Wall time: 16h 43min 35s


In [36]:
dfs = [gpd.read_file(file) for file in files]
dfs = pd.concat(dfs)
dfs.rename(columns={"local_NHDLength": "localNHD_idx", "local_ADWRLength": "localADWR_idx"},inplace=True)
dfs.to_file("./Data/AssessmentByHUC4/AllHUCs_LengthsComp_Arizona.gpkg", driver="GPKG")

In [56]:
def parseValues(v):
    if pd.isnull(v):
        idxes = [np.nan,np.nan,np.nan]
    elif "," not in v:
        idxes = [v,np.nan,np.nan]
    else:
        vs = v.split(",")[:3]
        if len(vs) == 3:
            idxes = vs
        elif len(vs) == 2:
            idxes = [vs[0], vs[1], np.nan]
        
        else:
            raise ValueError(f"Problem assigning values for {v}")
            
    return pd.Series(idxes)
            
dfs[["NHDidx_1", "NHDidx_2", "NHD_idx_3"]] = dfs.localNHD_idx.swifter.apply(lambda v: parseValues(v))
dfs[["ADWRidx_1", "ADWRidx_2", "ADWRidx_3"]] = dfs.localADWR_idx.swifter.apply(lambda v: parseValues(v))

C:\Users\BenJames\anaconda3\envs\arcgis\lib\site-packages\tqdm\std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [99]:
t = dfs.iloc[:10000].copy()

In [101]:
%%time
def getNHDLength(fidx):
    if pd.isnull(fidx) or fidx == None or fidx=="":
        return None
    try:
        l = nhdFlowlines_huc12[nhdFlowlines_huc12.Huc12FeatIdx == int(fidx)].geometry.length.values
        if len(l) > 1:
            raise ValueError
        else:
            return l[0]
    except:
        print(fidx, pd.isnull(fidx),  fidx == None, fidx=="")
        return None
 



Wall time: 11.3 s


In [143]:
dfs.NHDidx_1 = pd.to_numeric(dfs.NHDidx_1)
dfs.ADWRidx_1 = pd.to_numeric(dfs.ADWRidx_1)

In [160]:
decimal_precision = 3
dfs2 = pd.merge(dfs, nhdFlowlines_huc12[["Huc12FeatIdx", "geometry"]], left_on="NHDidx_1", right_on="Huc12FeatIdx", how="left", suffixes=("","_NHD"))
dfs2["localNHD_length"] = dfs2.geometry_NHD.apply(lambda g: round(g.length,decimal_precision) if g != None else np.nan)

dfs2 = pd.merge(dfs2, adwr_huc12[["Huc12FeatIdx", "geometry"]], left_on="ADWRidx_1", right_on="Huc12FeatIdx", how="left", suffixes=("","_ADWR"))
dfs2["localADWR_length"] = dfs2.geometry_ADWR.apply(lambda g: round(g.length, decimal_precision) if g != None else np.nan)
del dfs2["Huc12FeatIdx_NHD"], dfs2["geometry_NHD"], dfs2["geometry_ADWR"], dfs2["Huc12FeatIdx_ADWR"]

all_same = dfs2[(dfs2.localNHD_length==dfs2.geometry.length.round(decimal_precision)) & (dfs2.localADWR_length==dfs2.geometry.length.round(decimal_precision))]
num_same = len(all_same)
print(f"{num_same} total streams are identical across three datasets ({round(100*num_same/len(dfs),1)}%)")

In [59]:
dfs.to_file("./Data/AssessmentByHUC4/AllHUCs_LengthsComp_Arizona.gpkg", driver="GPKG")
chime.success()

In [165]:
not_same = dfs2[(dfs2.localNHD_length!=dfs2.geometry.length.round(decimal_precision)) | (dfs2.localADWR_length!=dfs2.geometry.length.round(decimal_precision))]
not_same.to_file("./Data/AssessmentByHUC4/AllHUCs_NotSame.gpkg", driver="GPKG")

In [74]:
threes = len(dfs[dfs.EditCode == 3])
zeros = len(dfs[dfs.EditCode == 0])
zeros_length = dfs[dfs.EditCode == 0].geometry.length.sum()
total_length = dfs.geometry.length.sum()

print(f"{zeros} streams of {len(dfs)} total streams are identical ({round(zeros/len(dfs)*100,2)}%)\n\t{round(100*zeros_length/total_length,2)}% by stream length")
#print(f"{threes} streams of {len(dfs)} total streams to be checked ({round(threes/len(dfs)*100,2)}%)")

310979 streams of 503575 total streams are identical (61.75%)
	57.1% by stream length


In [45]:
huc8s = gpd.read_file(r"S:\BHickson\Shared\NHD\NHD_H_Arizona_State_GDB.gdb", layer="WBDHU8")
huc8s = huc8s[(huc8s["HUC8"] == "15050301") | (huc8s["HUC8"] == "15050202")].to_crs(huc_15050301.crs)
huc8s

,TNMID,MetaSourceID,SourceDataDesc,SourceOriginator,SourceFeatureID,LoadDate,GNIS_ID,AreaAcres,AreaSqKm,States,HUC8,Name,Shape_Length,Shape_Area,geometry
40,{21A6AE9C-8C85-4578-AE5A-368093D0E9E5},None,None,None,None,2012-06-11T07:54:56,None,1587400.21,6423.99,"AZ,MX",15050202,Upper San Pedro,5.472283,0.610343,"MULTIPOLYGON (((-12252803.888 3804351.247, -12..."
52,{EE94F429-A5EE-4583-8558-00435EF322A7},None,None,None,None,2020-06-04T20:32:35,None,1680515.46,6800.81,"AZ,MX",15050301,Upper Santa Cruz,6.245822,0.647308,"MULTIPOLYGON (((-12341933.794 3850142.150, -12..."


In [46]:
huc_15050301 = gpd.read_file("./Data/AssessmentByHUC4/adeq_huc12_HUC4-1505.gpkg")

In [49]:
%%time
tdf = gpd.overlay(huc_15050301, huc8s)

tdf.to_file("./Data/PilotStreams.gpkg", driver="GPKG")

In [51]:
print(tdf.shape)
print(tdf[tdf.EditCode == 0].shape)

(38206, 39)
(24829, 39)
